In [67]:
import numpy as np
from random import gauss
from scipy.stats import norm
import scipy.misc
import math

class European:
    '''
    S: current stock price
    X: strike price
    r: risk-free rate
    sigma: volatility
    T: Time until maturity (in years)
    d: dividend yield (default assumption is 0)
    '''

    def __init__(self, S, X, r, sigma, T, d = 0):
        self.X = X
        self.S = float(S)
        self.r = r
        self.sigma = sigma
        self.T = T
        self.d = d
        self.b = self.r - self.d

    #MonteCarlo Pricing
    def MonteCarlo(self, option):
        if option.lower() == "call":
            simulations = 100000
            disc_factor = np.exp(-self.r * self.T)
            payoffs = []
            for i in range(simulations):
                S_t = self.S * np.exp((self.r - 0.5 * self.sigma**2) * self.T + self.sigma * np.sqrt(self.T) * gauss(0,1.0))
                payoffs.append(max(0.0, S_t - self.X))
            return disc_factor * (sum(payoffs) / float(simulations))
        elif option.lower() == "put":
            simulations = 100000
            disc_factor = np.exp(-self.r * self.T)
            payoffs = []
            for i in range(simulations):
                S_t = self.S * np.exp((self.r - 0.5 * self.sigma**2) * self.T + self.sigma * np.sqrt(self.T) * gauss(0,1.0))
                payoffs.append(max(0.0,self.X - S_t))
            return disc_factor * (sum(payoffs) / float(simulations))
        else:
            return "Invalid data"

    # BlackScholes Pricing
    def BlackScholes(self, option):
        d1 = (np.log(self.S/self.X) + ((self.r-self.d) + ((self.sigma**2)*0.5))*self.T) / (self.sigma*np.sqrt(self.T))
        d2 = d1 - self.sigma*np.sqrt(self.T)
        if option.lower() == "call":
            return self.S * np.exp(-self.d*self.T) * norm.cdf(d1) - self.X * np.exp(-self.r*self.T) * norm.cdf(d2)
        elif option.lower() == "put":
            return self.X * np.exp(-self.r*self.T) * norm.cdf(-d2) - self.S * np.exp(-self.d*self.T) * norm.cdf(-d1)
        else:
            return "Invalid data"

    def binomial(self, option):
        N = 100
        dt = self.T/N
        u =  math.exp(self.sigma * math.sqrt(dt))
        d = 1/u
        p = (math.exp(self.r * dt) - d)/(u - d)
        C = {}
        P = {}
        
        if option.lower() == "call":
            for m in range(0, N+1):
                    C[(N, m)] = max(self.S * (u ** (2*m - N)) - self.X, 0)
            for self.X in range(N-1, -1, -1):
                for m in range(0,self.X+1):
                    C[(self.X, m)] = math.exp(-self.r * dt) * (p * C[(self.X+1, m+1)] + (1-p) * C[(self.X+1, m)])
            return C[(0,0)]
        
        if option.lower() == "put":
            for m in range(0, N+1):
                    P[(N, m)] = max(self.X - self.S * (u ** (2*m - N)), 0)
            for self.X in range(N-1, -1, -1):
                for m in range(0,self.X+1):
                    P[(self.X, m)] = math.exp(-self.r * dt) * (p * P[(self.X+1, m+1)] + (1-p) * P[(self.X+1, m)])
            return P[(0,0)]

In [74]:
Test = European (55, 50, 0.08, 0.10,1)

In [75]:
Test.MonteCarlo("put")

0.08035286625959709

In [70]:
Test.BlackScholes('put')

0.08078989140076454

In [71]:
Test.binomial('put')

0.08088654060590277